# Getting Started with SQL and BigQuery
[Tutorial Link](https://www.kaggle.com/dansbecker/getting-started-with-sql-and-bigquery)

## Introduction
- **SQL**(Structured Query Language) - A programming language used with databases. <br>
- **BigQuery** - A web service that lets us apply SQL to huge datasets.

In this tutorial we will learn about accessing and examining BigQuery datasets.

## First BigQuery commands
To use BigQuery we will import the Python package.

In [ ]:
from google.cloud import bigquery

First step in the workflow is to create a `Client` objct. `Client` object plays a central role in retrieving information from bigquery datasets.

In [ ]:
# Create a client object
client = bigquery.Client()

We will work with dataset of posts on [Hacker News](https://news.ycombinator.com/)

In BigQuery each dataset in contained in a corresponding project. In this case `hacker_news` dataset is contained in the `bigquery-public-data` project. To access the dataset,
- Construct a reference to the dataset by using `dataset()` method.
- Next, use the `get_dataset()` method, along with the reference we just constructed, to fetch the dataset.

In [ ]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [ ]:
# construct a reference to the hacker_news dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Each dataset is a collection of tables.

We use `list_tables()` method to list tables present in the dataset.

In [ ]:
# List tables present in "hacker_news" dataset
tables = list(client.list_tables(dataset))

# print names of all tables present in the dataset
for table in tables:
    print(table.table_id)

Similar to how we fetched a dataset, we can fetch a table. In the code below we fetch the `full` table in the `hacker_news` dataset using `get_table()` method.

In [ ]:
# Create a table reference ("full")
table_ref = dataset_ref.table("full")

# API request - fetch the full table
table = client.get_table(table_ref)

What we have learnt so far:
<img src="https://i.imgur.com/biYqbUB.png"/>

## Table Schema
Structure of a table is called its schema. We need to understand a table's schema to pull the data we want.

Here we will investigate the `full` table that we fetched earlier.

In [ ]:
table.schema

Each `SchemaField` tells us about a specific column (also referred to as `field`). In order the information is:
- The **name** of the column
- The **field type** (or **data type**) of the column
- The **mode** of the column (`NULLABLE` means the column allows NULL values, default)
- A **description** of the data in that column

The first **field** has the `SchemaField`:
`SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", ()),` <br/>
This tells us:
- The **field** (or **column**) is called `by`,
- The data in the field is strings,
- This column allow NULL values,
- This column contains username of item's author.

We can use `list_rows()` method to show first five rows of the `full` table to make sure its right. This returns a BigQuery `RowIterator` object which can be converted to pandas DataFrame using `to_dataframe()` method.

In [ ]:
# print first five rows of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

Above code printed out first five rows of all **fields**, we can also print first five rows of selected fields if we want. For example here we will print first five rows of `by` **field**

In [ ]:
# print first five rows of "by" column (or field) of "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

**Setup**
- import `from google.cloud import bigquery`
- create a client object `client = bigquery.Client()

**Fetching Dataset**
- create a reference to dataset `dataset_ref = client.dataset("dataset_name", project="project_name")`
- fetch dataset `dataset = client.get_dataset(dataset_ref)

**List all tables present in dataset**
- list tables `tables = list(client.list_tables(dataset))` `print(table[0].table_id)`

**Fetch a table from dataset**
- create reference to a table `table_ref = dataset_ref.table("table_name")`
- fetch table `table = client.get_table(table_ref)`

**Schema**
- schema of a table `table.schema`

**List rows**
- all rows `client.list_rows(table, max_results=5).to_dataframe()`
- specific columns `client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()`

Exercises of this tutorial are solved [here](https://www.kaggle.com/mahendrabishnoi2/exercise-getting-started-with-sql-and-bigquery)